<a href="https://colab.research.google.com/github/UiinKim/Music_Generation/blob/main/LSTM_MusicGeneration_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.nn

!pip install --upgrade torch torchvision

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving 0fithos.mid to 0fithos.mid
Saving 8.mid to 8.mid
Saving ahead_on_our_way_piano.mid to ahead_on_our_way_piano.mid
Saving AT.mid to AT.mid
Saving balamb.mid to balamb.mid
Saving bcm.mid to bcm.mid
Saving BlueStone_LastDungeon.mid to BlueStone_LastDungeon.mid
Saving braska.mid to braska.mid
Saving caitsith.mid to caitsith.mid
Saving Cids.mid to Cids.mid
Saving cosmo.mid to cosmo.mid
Saving costadsol.mid to costadsol.mid
Saving dayafter.mid to dayafter.mid
Saving decisive.mid to decisive.mid
Saving dontbeafraid.mid to dontbeafraid.mid
Saving DOS.mid to DOS.mid
Saving electric_de_chocobo.mid to electric_de_chocobo.mid
Saving Eternal_Harvest.mid to Eternal_Harvest.mid
Saving EyesOnMePiano.mid to EyesOnMePiano.mid
Saving ff1battp.mid to ff1battp.mid
Saving FF3_Battle_(Piano).mid to FF3_Battle_(Piano).mid
Saving FF3_Third_Phase_Final_(Piano).mid to FF3_Third_Phase_Final_(Piano).mid
Saving FF4.mid to FF4.mid
Saving ff4_piano_collections-main_theme.mid to ff4_piano_collections-main_theme.

In [ ]:
from music21 import converter, instrument, note, chord
#import glob # jupyter 용

notes=[]

#for file in glob.glob('C:/Users/UiinKim/MusicGeneration/Classical-Piano-Composer-master/midi_songs/*.mid'):
for file in uploaded:
    midi=converter.parse(file) #converter은 음악을 로드한다. parse를 사용하여 파일의 item을 파싱하고 stream(객체)에 넣어준다
    notes_to_parse=None
    try:
        parts=instrument.partitionByInstrument(midi)#단일 stream, score, multi part 구조인 경우에 각각 악기별로 partition을 나누고 다른 파트들을 하나로 합쳐준다.(피아노로 합한다는 것 같음)
    except TypeError:
        print('## 1 {} file occur error.'.format(file))
    if parts:
        print('## 2 {} file has instrument parts'.format(file))
        notes_to_parse=parts.parts[0].recurse() #midi 라는 stream(객체) 안에 존재하는 iterable(순서가 존재할 수 있는 list tuple 등)한 객체(여기선 list)를 iterator로 만든다-> 순서가 생김
    else:
        print('## 3 {} file has notes in a flat structure'.format(file)) #평평한 구조이므로 그냥 추가
        notes_to_parse=midi.flat.notes #평평하게 음표를 넣는 것인듯
    for element in notes_to_parse:
        if isinstance(element, note.Note): #element가 note인 상태이면
            notes.append(str(element.pitch)) #문자열로 note 추가한다
        elif isinstance(element, chord.Chord): #element가 chord인 상태이면
            notes.append('.'.join(str(n)for n in element.normalOrder)) #문자열로 .으로 구분된 note들이 합해진 chord를 구분하여 note들로 추가

## 2 0fithos.mid file has instrument parts
## 2 8.mid file has instrument parts
## 2 ahead_on_our_way_piano.mid file has instrument parts
## 2 AT.mid file has instrument parts
## 2 balamb.mid file has instrument parts
## 2 bcm.mid file has instrument parts
## 2 BlueStone_LastDungeon.mid file has instrument parts
## 2 braska.mid file has instrument parts
## 2 caitsith.mid file has instrument parts
## 2 Cids.mid file has instrument parts
## 2 cosmo.mid file has instrument parts
## 2 costadsol.mid file has instrument parts
## 2 dayafter.mid file has instrument parts
## 2 decisive.mid file has instrument parts
## 2 dontbeafraid.mid file has instrument parts
## 2 DOS.mid file has instrument parts
## 2 electric_de_chocobo.mid file has instrument parts
## 2 Eternal_Harvest.mid file has instrument parts
## 2 EyesOnMePiano.mid file has instrument parts
## 2 ff1battp.mid file has instrument parts
## 2 FF3_Battle_(Piano).mid file has instrument parts
## 2 FF3_Third_Phase_Final_(Piano).mid file ha

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


## 2 ff7themep.mid file has instrument parts
## 2 FF8_Shuffle_or_boogie_pc.mid file has instrument parts
## 2 ff8-lfp.mid file has instrument parts
## 2 ff11_awakening_piano.mid file has instrument parts
## 2 FFIII_Edgar_And_Sabin_Piano.mid file has instrument parts
## 2 FFIX_Piano.mid file has instrument parts
## 2 FFIXQuMarshP.mid file has instrument parts
## 2 FFVII_BATTLE.mid file has instrument parts
## 2 FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid file has instrument parts
## 2 Fiend_Battle_(Piano).mid file has instrument parts
## 2 Fierce_Battle_(Piano).mid file has instrument parts
## 2 figaro.mid file has instrument parts
## 2 Final_Fantasy_7_-_Judgement_Day_Piano.mid file has instrument parts
## 2 Final_Fantasy_Matouyas_Cave_Piano.mid file has instrument parts
## 2 Finalfantasy5gilgameshp.mid file has instrument parts
## 2 Finalfantasy6fanfarecomplete.mid file has instrument parts
## 2 fortresscondor.mid file has instrument parts
## 2 Fyw_piano.mid file has instrumen

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


## 2 Rachel_Piano_tempofix.mid file has instrument parts
## 2 redwings.mid file has instrument parts
## 2 relmstheme-piano.mid file has instrument parts
## 2 roseofmay-piano.mid file has instrument parts
## 2 rufus.mid file has instrument parts
## 2 Rydia_pc.mid file has instrument parts
## 2 sandy.mid file has instrument parts
## 2 sera_.mid file has instrument parts
## 2 sobf.mid file has instrument parts
## 2 Still_Alive-1.mid file has instrument parts
## 2 Suteki_Da_Ne_(Piano_Version).mid file has instrument parts
## 2 thenightmarebegins.mid file has instrument parts
## 2 thoughts.mid file has instrument parts
## 2 tifap.mid file has instrument parts
## 2 tpirtsd-piano.mid file has instrument parts
## 2 traitor.mid file has instrument parts
## 2 ultimafro.mid file has instrument parts
## 2 ultros.mid file has instrument parts
## 2 VincentPiano.mid file has instrument parts
## 2 ViviinAlexandria.mid file has instrument parts
## 2 waltz_de_choco.mid file has instrument parts
## 2 z_a

In [ ]:
print(notes)

['4.9', 'E2', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '11.4', '4.9', '11.4', '4.9', '4.9', '4.9', '4.9', '4.9', '0.4', 'E2', '4.9', '0.4', '4.9', '4.9', '4.9', '4.9', '4.9', '9.2', '4.9', '9.2', '9.2', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', 'E2', '4.9', '4.9', '4.9', '4.9', '4.9', 'E5', '5.8', 'A5', '4.9', '4.9', '5.11', '4.9', '5.11', '4.9', '4.9', '4.9', 'E5', '5.8', 'A5', '4.9', '4.9', '9.0', 'E2', '4.9', '9.0', '4.9', '4.9', '4.9', 'E5', '5.8', 'A5', '4.9', '4.9', '11.2', '4.9', '11.2', '11.2', '4.9', '4.9', '4.9', 'E5', '5.8', 'A5', '4.9', '4.9', '3.7.11', 'E-2', '3.7.11', 'B2', 'G2', '1.5.9', 'F#2', '1.5.9', '3.7.11', 'E-2', '3.7.11', 'G2', 'B2', 'E-3', '1.5.9', 'G#2', '1.5.9', '1.5.9', 'F3', 'F2', 'F2', 'F2', 'F2', 'F2', '4.9', 'E5', '4.9', 'C5', '4.9', 'A5', '4.9', '5.9', 'F5', '5.9', 'C5', '5.9', 'A5', '5.9', '4.9', 'E5', '4.9', 'C5', '4.9', 'A5', '4.9', 'F5', '5.9', 'C5', '5.9', 'E5', '5.9', 'D5', '5.9', 'E5', '4.9', 'E-5', '4.9', 'B5', '4.9', '4.9', 'A5', '5.9', 

In [ ]:
import numpy as np
from tensorflow.python.keras.utils import np_utils
sequence_length=100 #음표 하나를 예측하는 데 사용하는 음표의 개수
#곡에 있는 모든 계이름의 이름을 pitchnames 변수에 저장 -> set(중복 제거)과 sorted(정렬->음표는 C, D등으로 정렬 가능)로 사전 만들기 준비

pitchnames=sorted(set(item for item in notes))
n_vocab=len(pitchnames)

#음표사전을 만든다
note_to_int=dict((note, number) for number, note in enumerate(pitchnames))
network_input=[]
network_output=[]

#입력 시퀀스 만들기
for i in range(0, len(notes)-sequence_length, 1): #하나의 note를 생성하는데 100개의 note를 소모하므로 i(생성되는 note)는 1씩 증가하고 마지막 길이는 최대 길이에서 시퀀스의 길이만큼 빼야함
    sequence_in=notes[i:i+sequence_length] #100개의 note를 사용하여 1개를 예측하고
    sequence_out=notes[i+sequence_length] #101번째의 note가 예측된 레이블 1개의 note
    network_input.append([note_to_int[char] for char in sequence_in]) #int로 바꿔서 모델에 넣을 input에 100개의 note를 가진 1개의 리스트로 추가한다
    network_output.append(note_to_int[sequence_out])#int로 바꿔서 1개의 note 레이블로 사용

n_patterns=len(network_input) #총 시퀀스의 개수

#데이터 입력 형태를 LSTM 레이어에 알맞게 변경
network_input=np.reshape(network_input, (n_patterns, sequence_length,1))
#괄호 안 설명 -> 입력으로 들어갈 1개의 시퀀스 당 100개의 note로 구성된 network_input을 총 시퀀스의 개수만큼의 note로 출력되고 한 note를 예측하는데(1을 뜻함->한 note) 사용하는 시퀀스의 길이는 100(한 시퀀스의 길이)로 설정한다.
#n_pattern은 1epoch 당 들어가는 배치의 크기(network_input의 총 시퀀스 수만큼 모두 1epoch당 들어간다)
#입력값을 normalizing(정규화)
network_input=network_input/float(n_vocab) #note_input을 음표사전의 길이로 나눈다 -> 정규화를 위함(데이터를 0~1사이로 스케일링 -> 이유는 입력 값이 가능한 모든 음표의 수에 대한 비율로 나누기 때문)-> 일정한 범위 이내에 있으므로 모델의 수렴 속도 향상 및 가중치 업데이트 안정화
network_output=np_utils.to_categorical(network_output) #one-hot encoding 10진->2진

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint

model=Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
#이 층에서 가지는 뉴런 수가 256개-> output_dim이다, 100개의 note가 들어오고 1개의 음표사전 중 있는 음을 나타내기 위해 1개의 note를 표현한다(음표의 특성 수), 시퀀스 출력을 반환한다.
model.add(Dropout(0.3)) #죽이는 뉴런 비율
model.add(LSTM(512, return_sequences=True)) #모델의 표현력 향상을 위해 512
model.add(Dropout(0.3))
model.add(LSTM(256)) #return_sequences가 없으므로 출력만 전달(Fully connected를 위함인듯)
model.add(Dense(256)) #Fully connected layer로 LSTM의 출력을 전달받아 추가적인 표현 가능
model.add(Dropout(0.3))
model.add(Dense(n_vocab))#신경망의 출력을 음표의 확률 분포로 변환 -> n_vocab의 이유는 음표 사전에 존재하는 음표들로만 출력이 되야하기 때문
model.add(Activation('softmax')) #음표가 속할 확률을 출력
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#두 개 이상의 카테고리 존재 -> -categorical_crossentropy, 순환신경망에서 자주 사용하는 rmsprop

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath='weights-improvement-{epoch:02d}-{loss:4f}-bigger.hdf5'
#모델 가중치를 저장할 파일 경로
checkpoint=ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
#모델 체크포인트는 매 에포크마다 네트워크 노드의 가중치를 파일에 저장하는 방법, 기준은 손실이고 가장 작은 값의 모델의 구조와 가중치만 저장
callbacks_list=[checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#입력 시퀀스, 출력 시퀀스, 200번 반복, 64개의 샘플로 가중치 갱신,

Epoch 1/200
717/717 [==============================] - 35s 43ms/step - loss: 4.6195
Epoch 2/200
  1/717 [..............................] - ETA: 29s - loss: 4.4188

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


717/717 [==============================] - 32s 44ms/step - loss: 4.5222
Epoch 3/200
717/717 [==============================] - 32s 45ms/step - loss: 4.4890
Epoch 4/200
717/717 [==============================] - 33s 47ms/step - loss: 4.4572
Epoch 5/200
717/717 [==============================] - 34s 48ms/step - loss: 4.4224
Epoch 6/200
717/717 [==============================] - 35s 48ms/step - loss: 4.3773
Epoch 7/200
717/717 [==============================] - 34s 47ms/step - loss: 4.3307
Epoch 8/200
717/717 [==============================] - 35s 49ms/step - loss: 4.2529
Epoch 9/200
717/717 [==============================] - 34s 47ms/step - loss: 4.1623
Epoch 10/200
717/717 [==============================] - 34s 48ms/step - loss: 4.0669
Epoch 11/200
717/717 [==============================] - 34s 47ms/step - loss: 3.9677
Epoch 12/200
717/717 [==============================] - 34s 47ms/step - loss: 3.8519
Epoch 13/200
717/717 [==============================] - 34s 48ms/step - loss: 3.7210


In [ ]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인
# '/content' 디렉토리에 있는 파일 목록 확인
files = os.listdir('/content')
print(files)

/content
['.config', 'z_aeristhemepiano.mid', 'dontbeafraid.mid', 'gerudo.mid', 'weights-improvement-05-3.253520-bigger.hdf5', 'Gold_Silver_Rival_Battle.mid', 'weights-improvement-09-4.191451-bigger.hdf5', 'great_war.mid', 'weights-improvement-47-0.937721-bigger.hdf5', 'thenightmarebegins.mid', 'weights-improvement-15-2.348154-bigger.hdf5', 'rufus.mid', 'weights-improvement-19-2.077695-bigger.hdf5', 'Finalfantasy5gilgameshp.mid', 'relmstheme-piano.mid', 'ff1battp.mid', 'balamb.mid', 'weights-improvement-81-0.608602-bigger.hdf5', 'sobf.mid', 'weights-improvement-44-1.009022-bigger.hdf5', 'weights-improvement-65-0.700844-bigger.hdf5', 'Ff7-Jenova_Absolute.mid', 'weights-improvement-63-0.722145-bigger.hdf5', 'ultimafro.mid', 'weights-improvement-76-0.622068-bigger.hdf5', 'caitsith.mid', 'weights-improvement-80-0.611151-bigger.hdf5', 'weights-improvement-34-1.322837-bigger.hdf5', 'ultros.mid', 'weights-improvement-08-2.944376-bigger.hdf5', '0fithos.mid', 'weights-improvement-60-0.750970-bi

In [ ]:
#음악 생성
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# 각각의 뉴런(노드)의 가중치를 로드합니다.
# 파일에 저장한 학습 결과를 가져오는 것과 같습니다!
model.load_weights('weights.hdf5') #가중치 어떤 파일 가져왔는지 물어보기

In [ ]:
# 입력 시퀀스 랜덤
start = np.random.randint(0, len(network_input) - 1)

# 숫자 -> 노트 매핑 사전 생성
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
#random으로 구한 index를 사용하여 input으로 들어간 network의 start번째 시퀀스를 pattern으로 지정한다.

prediction_output = []

# 500개의 note 생성
for note_index in range(500):
    prediction_input = np.array(pattern).reshape(1, len(pattern), 1)
    # LSTM 구조로 변경. 1의 배치사이즈(1 epoch 당 한 개의 note를 나타내는 한 시퀀스(network_input[0])가 들어간다), 한 시퀀스의 길이(100)만큼의 길이가 들어가고, 1개의 note를 나타낸다.
    #결국 1epoch 당 한개의 note를 나타내기 위해 한개의 시퀀스가 들어가는 느낌
    prediction_input = prediction_input / float(n_vocab) #0과 1사이로 표현

    # 입력값에 대해 다음 note 예측
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)  # 예측값들 중 가장 예측값(softmax 활용)이 높은 인덱스를 선정

    # 결과를 int->note로 변경
    result = int_to_note[index]
    prediction_output.append(result) #output에 string으로 바뀐 예측 note를 저장

    pattern = np.append(pattern[1:], index) #pattern의 0이 아닌 1부터 인덱스를 다음 인덱스로 옮겨 다음 100개로 다음 음표 예측 준비


In [ ]:
offset=0 #객체의 위치 표현
output_notes=[]
#모델에 의해 예측된 값을 바탕으로 노트와 코드 객체를 만든다
for pattern in prediction_output:
  #패턴(출력값)이 chord일때
  if('.'in pattern) or pattern.isdigit(): # '.'이 있거나 문자열에 숫자인 문자만 있을 경우
    notes_in_chord=pattern.split('.')
    notes=[]
    for current_note in notes_in_chord: #코드로 판별된 chord들
      new_note = note.Note(int(current_note)) #chord를 정수로 바꾼 후 note 객체로 바꿔준다.(코드를 구성하는 개별 note들)
      new_note.storedInstrument=instrument.Piano() #piano 형식으로 바꾼다
      notes.append(new_note) #note객체들을 notes 리스트에 추가
    new_chord=chord.Chord(notes) #notes에 있는 note들을 하나의 코드로 묶는다
    new_chord.offset=offset #new_chord 객체의 offset 속성을 현재의 offset으로 설정한다.-> 코드 객체의 타이밍 조절
    output_notes.append(new_chord) #최종 리스트에 코드 추가
  #패턴(출력값)이 note일때
  else:
    new_note=note.Note(pattern) #바로 객체 생성
    new_note.offset=offset #note 객체의 offset을 현재 offset으로 설정
    new_note.storeInstrument=instrument.Piano() #피아노로 변경
    output_notes.append(new_note) #최종 리스트에 note 객체 추가

  #각 반복마다 오프셋을 0.5씩 증가 -> 만약 증가시키지 않으면 같은 오프셋(위치)에 음이 쌓이게 된다
  offset+=0.5

In [ ]:
from music21 import stream

midi_stream=stream.Stream(output_notes) #하나의 Stream으로 리스트를 변경
midi_stream.write('midi', fp='test_output.mid') #midifile로 작성

'test_output.mid'